In [45]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import imgaug.augmenters as iaa

DATASET_DIR = "dataset"   # folder inside backend/
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20

CLASSES = ["bio-degradable", "non-biodegradable"]


In [35]:
augmenter = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Affine(rotate=(-15, 15)),
    iaa.Multiply((0.85, 1.15)),
    iaa.GaussianBlur(sigma=0.3),
    iaa.AdditiveGaussianNoise(scale=0.01 * 255),
    iaa.LinearContrast((0.8, 1.2)),
])


In [36]:
class ImgAugSequence(tf.keras.utils.Sequence):
    def __init__(self, keras_generator, augmenter):
        self.generator = keras_generator
        self.augmenter = augmenter

    def __len__(self):
        return len(self.generator)

    def __getitem__(self, idx):
        images, labels = self.generator[idx]
        # Apply IAG augmentations
        images = self.augmenter(images=images)
        return images, labels

In [38]:
train_gen = ImageDataGenerator(
    validation_split=0.3,
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

train_keras = train_gen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    subset="training"
)

train_data = ImgAugSequence(train_keras, augmenter)  # <--- ImgAug here

val_data = train_gen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    subset="validation"
)

Found 2696 images belonging to 2 classes.
Found 1154 images belonging to 2 classes.


In [39]:
labels = train_keras.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
cw = {i: weight for i, weight in enumerate(class_weights)}
print("\nClass Weights:", cw)


Class Weights: {0: 1.1423728813559322, 1: 0.8891820580474934}


In [40]:
base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False   # <--- DO NOT UNFREEZE ANYTHING

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = tf.keras.applications.efficientnet.preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)


In [41]:

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,213,797 (16.07 MB)

 Trainable params: 164,226 (641.51 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [42]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    class_weight=cw
)

/Users/suryanshagarwal/Smart-Waste-Segregation-System/.venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 35s 358ms/step - accuracy: 0.7070 - loss: 0.5766 - val_accuracy: 0.9211 - val_loss: 0.2801
Epoch 2/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 28s 334ms/step - accuracy: 0.7923 - loss: 0.4550 - val_accuracy: 0.9315 - val_loss: 0.2350
Epoch 3/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 31s 363ms/step - accuracy: 0.8153 - loss: 0.4155 - val_accuracy: 0.9272 - val_loss: 0.2198
Epoch 4/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 30s 359ms/step - accuracy: 0.8082 - loss: 0.4125 - val_accuracy: 0.9289 - val_loss: 0.2019
Epoch 5/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 34s 399ms/step - accuracy: 0.8131 - loss: 0.4031 - val_accuracy: 0.9289 - val_loss: 0.1967
Epoch 6/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 40s 466ms/step - accuracy: 0.8179 - loss: 0.3835 - val_accuracy: 0.9411 - val_loss: 0.1907
Epoch 7/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 43s 499ms/step - accuracy: 0.8238 - loss: 0.3808 - val_accuracy: 0.9341 - val_loss: 0.1899
Epoch 8/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 46s 537ms/step - accuracy: 0.8260 - loss: 0.3766 - val_accu

In [44]:
# Save .h5 for API
model.save("waste_model.h5")
print("✔ Saved waste_model.h5")

# Save SavedModel for TFLite
tf.saved_model.save(model, "saved_model")
print("✔ Exported SavedModel to saved_model/")


✔ Saved waste_model.h5
INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


✔ Exported SavedModel to saved_model/
